# BB84 Quantum Key Distribution (QKD) Protocol

This notebook is a _demonstration_ of the BB84 Protocol for QKD using Qiskit. 
BB84 is a quantum key distribution scheme developed by Charles Bennett and Gilles Brassard in 1984 ([paper]).
The first three sections of the paper are readable and should give you all the necessary information required. 

[paper]: http://researcher.watson.ibm.com/researcher/files/us-bennetc/BB84highest.pdf 


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import *

## Choosing bases and encoding states

Alice generates two binary strings. One encodes the basis for each qubit:

$0 \rightarrow$ Computational basis

$1  \rightarrow$ Hadamard basis

The other encodes the state:

$0  \rightarrow|0\rangle$ or $|+\rangle $ 

$1  \rightarrow|1\rangle$  or  $|-\rangle $ 

Bob also generates a binary string and uses the same convention to choose a basis for measurement


In [3]:
num_qubits = 32 


alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)


print(alice_state)
print(alice_basis)
print(bob_basis)

[0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 1 0 1 0 1 1 0]
[0 1 0 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 1 1]
[0 0 0 1 1 1 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 1 1 1 1 0 0 0 1 1 0 1]


## Creating the circuit

Based on the following results:

$X|0\rangle = |1\rangle$

$H|0\rangle = |+\rangle$

$ HX|0\rangle = |-\rangle$

Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode 1 in a qubit, she applies an $X$ gate to the qubit. To encode 0, no action is needed.
2. Wherever she wants to encode it in the Hadamard basis, she applies an $H$ gate. No action is necessary to encode a qubit in the computational basis.

3. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

4. Bob measures the qubits according to his binary string. To measure a qubit in the Hadamard basis, he applies an $H$ gate to the corresponding qubit and then performs a mesurement on the computational basis. 



In [21]:
bb84_circ = QuantumCircuit(num_qubits)

# Alice prepares her qubits
for index in range(len(alice_basis)):
    if alice_state[index] == 1:
        bb84_circ.x(index)
    if alice_basis[index] == 1:
        bb84_circ.h(index)
bb84_circ.barrier()  

# Bob measures the received qubits
for index in range(len(bob_basis)):
    if bob_basis[index] == 1:
        bb84_circ.h(index)
        
bb84_circ.barrier()        
bb84_circ.measure_all()
    
            
# bb84_circ.draw('mpl')


## Performing the actual experiment

Now we will run this simulation experiment on the `QasmSimulator()` backend.

In [20]:
job = execute(bb84_circ.reverse_bits(),backend=QasmSimulator(),shots=10)
counts = job.result().get_counts()

# Prints out all the measured binary strings by Bob (reversed)
for count in [*counts]:
    print(count) 

00101000011000000001110011111100
00101000011010001001110010110110
00101000011010001001110100110100
00110000111000001001110101010110
00111000011010000001100001111110
00111000011100001001110101010110
00111000111000001001110011111100
01101000011110000001110111111110
01111000111010000001110110010100


## Creating the key

Alice and Bob only keep the bits where their bases match

In [15]:
keys = []
for temp_key in [*counts]:
    key = ''
    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]: # Only choose bits where Alice and Bob chose the same basis
             key += str(temp_key[i])
    keys.append(key)
#     print(key)

Let us look at the length of the key

In [16]:
print(len(keys[0]))

18


In [17]:
for key in keys:
    print(key)

010001100000110110
010001100000110110
010001100000110110
010001100000110110
010001100000110110
010001100000110110
010001100000110110
010001100000110110
010001100000110110
010001100000110110
